In [ ]:
import os
import hashlib
import re
import shutil
from tkinter import *
from tkinter import filedialog, messagebox, simpledialog
from PIL import Image, ImageTk
from Crypto.Cipher import AES
from Crypto.Util.Padding import pad, unpad
from Crypto.Random import get_random_bytes
from captcha.image import ImageCaptcha
import random
import string
import mysql.connector


# Constants
UPLOAD_FOLDER = r"C:\Users\harik\OneDrive\Desktop\cyberattack"
os.makedirs(UPLOAD_FOLDER, exist_ok=True)

# Dummy login credentials
DUMMY_EMAIL = "admin@example.com"
DUMMY_PASSWORD = "admin123"

# Database connection
db_config = {
    'host': 'localhost',
    'user': 'root',
    'password': 'harish@26',  # Replace with your MySQL password
    'database': 'SecureFileManager'
}
conn = mysql.connector.connect(**db_config)
cursor = conn.cursor()

# Main Application Window
main_app = Tk()
main_app.title("Secure File Manager")
main_app.geometry("800x600")
main_app.configure(bg="white")
main_app.resizable(False, False)

# Function to display the login page
def create_login_page():
    def validate_login():
        email = email_entry.get().strip()
        password = password_entry.get().strip()

        if email == DUMMY_EMAIL and password == DUMMY_PASSWORD:
            messagebox.showinfo("Login Successful", "Welcome to the Admin Page!")
            login_window.destroy()
            open_admin_page()
        else:
            messagebox.showerror("Login Failed", "Invalid email or password!")

    login_window = Tk()
    login_window.title("Login Page")
    login_window.geometry("400x300")
    login_window.configure(bg="white")

    Label(login_window, text="Login", font=("Arial", 18), bg="white").pack(pady=20)

    frame = Frame(login_window, bg="white")
    frame.pack(pady=10)

    Label(frame, text="Email:", font=("Arial", 12), bg="white").grid(row=0, column=0, pady=5, padx=5, sticky="w")
    email_entry = Entry(frame, font=("Arial", 12), width=25)
    email_entry.grid(row=0, column=1, pady=5, padx=5)

    Label(frame, text="Password:", font=("Arial", 12), bg="white").grid(row=1, column=0, pady=5, padx=5, sticky="w")
    password_entry = Entry(frame, font=("Arial", 12), show="*", width=25)
    password_entry.grid(row=1, column=1, pady=5, padx=5)

    Button(login_window, text="Login", font=("Arial", 14), command=validate_login).pack(pady=20)

    login_window.mainloop()

# Function to display the admin page (Server page)
def open_admin_page():
    def display_files(folder_path): 
        """Display all files and subfolders in the specified folder."""
        for widget in main_frame.winfo_children():
            widget.destroy()  # Clear previous contents

        Label(main_frame, text=f"Files in: {folder_path}", font=("Arial", 14), bg="white").pack(pady=10)

        if not os.path.exists(folder_path):
            Label(main_frame, text="The folder does not exist.", font=("Arial", 12), bg="white").pack(pady=10)
            return

        items = os.listdir(folder_path)
        if not items:
            Label(main_frame, text="No files or folders found.", font=("Arial", 12), bg="white").pack(pady=10)
            return

        for item_name in items:
            full_path = os.path.join(folder_path, item_name)
            item_frame = Frame(main_frame, bg="white")
            item_frame.pack(pady=5, padx=10, fill="x")

            item_label = Label(item_frame, text=item_name, font=("Arial", 12), bg="white", relief="solid", anchor="w")
            item_label.pack(side="left", fill="x", expand=True)

            if os.path.isdir(full_path):
                Button(
                    item_frame,
                    text="Open",
                    font=("Arial", 10),
                    command=lambda path=full_path: display_files(path)
                ).pack(side="right", padx=10)
            else:
                Button(
                    item_frame,
                    text="View",
                    font=("Arial", 10),
                    command=lambda path=full_path: open_file(path)
                ).pack(side="right", padx=5)

                Button(
                    item_frame,
                    text="Decrypt",
                    font=("Arial", 10),
                    command=lambda path=full_path: decrypt_file(path)
                ).pack(side="right", padx=5)

    
    def open_file(file_path):
        """Open the selected file."""
        try:
            os.startfile(file_path)
        except Exception as e:
            messagebox.showerror("Error", f"Failed to open file: {e}")

    def decrypt_file(file_path):
        """Decrypt a file (text or image) using AES."""
        save_path = filedialog.asksaveasfilename(
            title="Save Decrypted File",
            defaultextension="",
            filetypes=[("All Files", "*.*"), ("Text Files", "*.txt"), ("Image Files", "*.jpg *.png")]
        )

        if not save_path:
            return  # User canceled the save operation

        passphrase = simpledialog.askstring("Decryption Key", "Enter the decryption key:", show="*")
        if not passphrase:
            messagebox.showerror("Error", "Decryption key is required.")
            return

        key = hashlib.sha256(passphrase.encode()).digest()

        try:
            with open(file_path, 'rb') as f:
                encrypted_data = f.read()

            iv = encrypted_data[:16]
            cipher = AES.new(key, AES.MODE_CBC, iv)
            decrypted_data = unpad(cipher.decrypt(encrypted_data[16:]), AES.block_size)

            with open(save_path, 'wb') as f:
                f.write(decrypted_data)
            os.startfile(save_path)
            messagebox.showinfo("Success", f"File decrypted and saved to: {save_path}")
        except Exception as e:
            messagebox.showerror("Error", f"Decryption failed: {e}")

    admin_window = Toplevel()
    admin_window.title("Admin Page")
    admin_window.geometry("800x600")
    admin_window.configure(bg="white")

    Label(admin_window, text="Admin Page", font=("Arial", 16), bg="white").pack(pady=10)
    Button(admin_window, text="Show Files", font=("Arial", 14), command=lambda: display_files(UPLOAD_FOLDER)).pack(pady=20)

    main_frame = Frame(admin_window, bg="white")
    main_frame.pack(fill="both", expand=True, padx=10, pady=10)


# Function to display the user page
def open_user_page():
    main_app.withdraw()  # Hide the main application window
    user_window = Toplevel()
    user_window.title("User Page")
    user_window.geometry("800x600")
    user_window.configure(bg="white")

    UPLOAD_FOLDER = r"C:\Users\harik\OneDrive\Desktop\cyberattack"
    os.makedirs(UPLOAD_FOLDER, exist_ok=True)

    db_config = {
        'host': 'localhost',
        'user': 'root',
        'password': 'harish@26',  # Replace with your MySQL password
        'database': 'SecureFileManager',
    }
    conn = mysql.connector.connect(**db_config)
    cursor = conn.cursor()

    captcha_text = ""

    # Function to generate CAPTCHA
    def generate_captcha():
        nonlocal captcha_text
        image_captcha = ImageCaptcha()
        captcha_text = ''.join(random.choices(string.ascii_letters + string.digits, k=6))
        captcha_image = image_captcha.generate_image(captcha_text)
        captcha_path = "captcha.png"
        captcha_image.save(captcha_path)
        return captcha_path

    # Function to refresh CAPTCHA
    def refresh_captcha():
        captcha_path = generate_captcha()
        img = Image.open(captcha_path)
        img = img.resize((150, 50), Image.Resampling.LANCZOS)
        captcha_photo = ImageTk.PhotoImage(img)
        captcha_label.configure(image=captcha_photo)
        captcha_label.image = captcha_photo

    # Function to validate sign-in
    def validate_sign_in():
        email = e2.get().strip()
        password = e3.get().strip()
        entered_captcha = captcha_entry.get().strip()

        if not email or not password or not entered_captcha:
            messagebox.showerror("Error", "All fields are required!")
            return

        if entered_captcha != captcha_text:
            messagebox.showerror("Error", "Invalid CAPTCHA. Please try again.")
            refresh_captcha()
            return

        try:
            query = "SELECT * FROM users WHERE email = %s AND password = %s"
            cursor.execute(query, (email, password))
            user = cursor.fetchone()
            if user:
                open_file_upload_screen(email)
            else:
                messagebox.showerror("Error", "Invalid email or password.")
        except Exception as e:
            messagebox.showerror("Database Error", str(e))

    # Function to open file upload screen
    def open_file_upload_screen(user_email):
        user_window.withdraw()  # Hide the user window
        file_screen = Toplevel()
        file_screen.title("File Management")
        file_screen.geometry("800x600")
        file_screen.configure(bg="white")

        def close_file_screen():
            file_screen.destroy()
            user_window.deiconify()  # Show the user window again

        def upload_file():
            file_path = filedialog.askopenfilename(title="Select a File", filetypes=[("All Files", "*.*")])
            if file_path:
                try:
                    filename = os.path.basename(file_path)
                    user_folder = os.path.join(UPLOAD_FOLDER, user_email)
                    os.makedirs(user_folder, exist_ok=True)
                    dest_path = os.path.join(user_folder, filename)
                    shutil.copy(file_path, dest_path)
                    messagebox.showinfo("Success", f"File uploaded successfully to {user_folder}!")
                    update_file_lists()
                except Exception as e:
                    messagebox.showerror("Error", f"Failed to upload file: {e}")

        def update_file_lists():
            file_list.delete(0, END)
            image_list.delete(0, END)
            user_folder = os.path.join(UPLOAD_FOLDER, user_email)
            if os.path.exists(user_folder):
                for file in os.listdir(user_folder):
                    if file.lower().endswith((".png", ".jpg", ".jpeg", ".bmp", ".gif", ".webp", ".pdf")):
                        image_list.insert(END, file)
                    else:
                        file_list.insert(END, file)

        def view_uploaded_file():
            selected = file_list.curselection()
            if not selected:
                messagebox.showwarning("No Selection", "Please select a file to view.")
                return
            selected_file = file_list.get(selected)
            full_path = os.path.join(UPLOAD_FOLDER, user_email, selected_file)
            messagebox.showinfo("File Path", f"Full Path:\n{full_path}")

        def view_image():
            selected = image_list.curselection()
            if not selected:
                messagebox.showwarning("No Selection", "Please select an image to view.")
                return
            selected_file = image_list.get(selected)
            full_path = os.path.join(UPLOAD_FOLDER, user_email, selected_file)

            image_window = Toplevel(file_screen)
            image_window.title("View Image")
            image_window.geometry("800x600")
            image_window.configure(bg="white")

            try:
                img = Image.open(full_path)
                img = img.resize((800, 600), Image.Resampling.LANCZOS)
                photo = ImageTk.PhotoImage(img)

                img_label = Label(image_window, image=photo, bg="white")
                img_label.image = photo
                img_label.pack(expand=True)
            except Exception as e:
                messagebox.showerror("Error", f"Cannot display image: {e}")

        def encrypt_file():
            selected_file = None
            file_path = None

        # Check file list and image list for selection
            if file_list.curselection():
                selected_index = file_list.curselection()[0]
                selected_file = file_list.get(selected_index)
                file_path = os.path.join(UPLOAD_FOLDER, user_email, selected_file)
            elif image_list.curselection():
                selected_index = image_list.curselection()[0]
                selected_file = image_list.get(selected_index)
                file_path = os.path.join(UPLOAD_FOLDER, user_email, selected_file)

            if not file_path:
                messagebox.showwarning("No Selection", "Please select a file or image to encrypt.")
                return

            pass_phrase = encrypt_pass_entry.get()
            if not pass_phrase:
                messagebox.showwarning("Passphrase Required", "Please enter a passphrase for encryption.")
                return

            key = hashlib.sha256(pass_phrase.encode()).digest()
            iv = get_random_bytes(16)

            try:
                with open(file_path, 'rb') as f:
                    file_data = f.read()

                padded_data = pad(file_data, AES.block_size)
                cipher = AES.new(key, AES.MODE_CBC, iv)
                encrypted_data = iv + cipher.encrypt(padded_data)

                user_folder = os.path.join(UPLOAD_FOLDER, user_email)
                os.makedirs(user_folder, exist_ok=True)
                save_path = os.path.join(user_folder, f"{selected_file}.enc")
                with open(save_path, 'wb') as f:
                    f.write(encrypted_data)
                    messagebox.showinfo("Success", f"Encrypted file saved to '{save_path}'")
            except Exception as e:
                messagebox.showerror("Error", f"Encryption failed: {e}")
    # File Management UI
        Label(file_screen, text=f"Welcome, {user_email}", font=("Arial", 14), bg="white").pack(pady=10)
        Button(file_screen, text="Upload File", command=upload_file).pack(pady=10)

    # Uploaded Files
        Label(file_screen, text="Uploaded Files:", font=("Arial", 12), bg="white").pack(pady=5)
        file_list = Listbox(file_screen, width=50, height=5)
        file_list.pack(pady=5)
        Button(file_screen, text="View Selected File", command=view_uploaded_file).pack(pady=5)

    # Uploaded Images
        Label(file_screen, text="Uploaded Images:", font=("Arial", 12), bg="white").pack(pady=5)
        image_list = Listbox(file_screen, width=50, height=5)
        image_list.pack(pady=5)
        Button(file_screen, text="View Selected Image", command=view_image).pack(pady=5)

    # Encryption Section
        encryption_frame = Frame(file_screen, bg="white")
        encryption_frame.pack(pady=10)

        Label(encryption_frame, text="Passphrase for Encryption:", bg="white").grid(row=0, column=0, padx=5, pady=5)
        encrypt_pass_entry = Entry(encryption_frame, show="*", width=30)
        encrypt_pass_entry.grid(row=0, column=1, padx=5, pady=5)
        Button(encryption_frame, text="Encrypt Selected File", command=encrypt_file, bg="maroon", fg="white").grid(row=0, column=2, padx=5, pady=5)

        update_file_lists()

    # Handle window close
        file_screen.protocol("WM_DELETE_WINDOW", close_file_screen)



    def register():
    # Implemented as before with security improvements
        reg_window = Toplevel(user_window)
        reg_window.title("Register")
        reg_window.geometry("800x600")
        reg_window.configure(bg="white")

        def display():
            email = reg_email.get()
            passe = reg_password.get()
            cpass = reg_confirm_password.get()
            first_name = e1.get()
            last_name = e10.get()

            if "@gmail.com" in email and len(passe) > 6:
                if re.match(r"^(?=.*[A-Za-z])(?=.*\d)(?=.*[@$!%*?&])[A-Za-z\d@$!%*?&]+$", passe):
                    if passe == cpass:
                        try:
                            cursor.execute(
                                "INSERT INTO users (email, password, first_name, last_name) VALUES (%s, %s, %s, %s)",
                                (email, passe, first_name, last_name)
                            )
                            conn.commit()
                            messagebox.showinfo("Success", "Account created successfully!")
                        except mysql.connector.IntegrityError:
                            messagebox.showwarning("Error", "Email already registered.")
                        except Exception as e:
                            messagebox.showerror("Error", f"An error occurred: {e}")
                    else:
                        messagebox.showwarning("Error", "Passwords do not match.")
                else:
                    messagebox.showwarning(
                        "Weak Password",
                         "Password should contain at least one letter, one digit, and one special character.")
            else:
                messagebox.showwarning("Invalid Details", "Please provide a valid email and strong password.")
        l1 = Label(reg_window, text="FIRST NAME", font=("ariel", 10), bg="white")
        l1.place(x=150, y=50)
        e1 = Entry(reg_window)
        e1.place(x=300, y=50, width=200)

        l10 = Label(reg_window, text="SECOND NAME", font=("ariel", 10), bg="white")
        l10.place(x=150, y=100)
        e10 = Entry(reg_window)
        e10.place(x=300, y=100, width=200)

        Label(reg_window, text="EMAIL", font=("Arial", 10), bg="white").place(x=150, y=150)
        reg_email = Entry(reg_window)
        reg_email.place(x=300, y=150, width=200)

        Label(reg_window, text="NEW PASSWORD", font=("Arial", 10), bg="white").place(x=150, y=200)
        reg_password = Entry(reg_window, show="*", bg="white", fg="black")
        reg_password.place(x=300, y=200, width=200)

        Label(reg_window, text="CONFIRM PASSWORD", font=("Arial", 10), bg="white").place(x=150, y=250)
        reg_confirm_password = Entry(reg_window, show="*", bg="white", fg="black")
        reg_confirm_password.place(x=300, y=250, width=200)

        Button(reg_window, text="CREATE ACCOUNT", command=display).place(x=250, y=350, width=200, height=30)


    # UI Elements for Sign-In
    Label(user_window, text="SIGN IN", font=("Arial", 15), bg="white").place(x=350, y=30)

    Label(user_window, text="EMAIL", font=("Arial", 10), bg="white").place(x=200, y=100)
    e2 = Entry(user_window, font=("Arial", 10))
    e2.place(x=350, y=100, width=200)

    Label(user_window, text="PASSWORD", font=("Arial", 10), bg="white").place(x=200, y=150)
    e3 = Entry(user_window, font=("Arial", 10), show="*")
    e3.place(x=350, y=150, width=200)

    Label(user_window, text="CAPTCHA", font=("Arial", 10), bg="white").place(x=200, y=210)
    captcha_label = Label(user_window, bg="white")
    captcha_label.place(x=350, y=210, width=150, height=50)

    captcha_entry = Entry(user_window, font=("Arial", 10))
    captcha_entry.place(x=350, y=270, width=150)

    Button(user_window, text="Refresh CAPTCHA", font=("Arial", 10), command=refresh_captcha).place(x=520, y=220)
    Button(user_window, text="Sign In", font=("Arial", 12), command=validate_sign_in).place(x=350, y=320)

    # Generate initial CAPTCHA
    refresh_captcha()
    
    Label(user_window, text="I don't have an account", font=("Arial", 7), bg="white").place(x=300, y=370)
    Button(user_window, text="REGISTER", font=("Arial", 7), command=register, bg="green", fg="white").place(x=450, y=370, width=80, height=15)


    user_window.mainloop()


# Main Screen Layout
Label(main_app, text="Secure File Manager", font=("Arial", 20), bg="white").pack(pady=50)

Button(main_app, text="Admin", font=("Arial", 14), command=create_login_page, bg="blue", fg="white").pack(pady=20)
Button(main_app, text="User", font=("Arial", 14),command =open_user_page, bg="green", fg="white").pack(pady=20)

main_app.mainloop()
